In [1]:
# Import libraries
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tqdm import tqdm
import gc

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", tf.test.gpu_device_name())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

2025-03-28 23:25:44.812386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743229544.847949   14786 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743229544.858578   14786 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 23:25:44.913640: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 1
TensorFlow is using: /device:GPU:0
GPU memory growth enabled.


I0000 00:00:1743229548.035627   14786 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1743229548.037005   14786 gpu_device.cc:2022] Created device /device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


In [2]:
def load_data(spectrogram_dir, label_dir, filter_piano=False):
    spectrograms = []
    labels = []

    # Load spectrograms
    spectrogram_files = sorted(os.listdir(spectrogram_dir))
    label_files = sorted(os.listdir(label_dir))

    total_files = len(spectrogram_files)

    for spec_file, label_file in tqdm(zip(spectrogram_files, label_files), 
                                      total=total_files, 
                                      desc="Loading Data", 
                                      unit="file"):
        # Load data
        spec_data = np.load(os.path.join(spectrogram_dir, spec_file))  # Shape: (n, 32, 42)
        label_data = np.load(os.path.join(label_dir, label_file)).T    # Shape: (n, 5)

        if filter_piano:
            # Extract only segments where piano (first row) is active
            piano_active_indices = np.where(label_data[:, 0] == 1)[0]
            spec_data = spec_data[piano_active_indices]
            label_data = label_data[piano_active_indices]

        # Append if there is valid data
        if len(spec_data) > 0:
            spectrograms.append(spec_data)
            labels.append(label_data)

    # Convert to numpy arrays 
    if spectrograms:
        X = np.vstack(spectrograms)
        Y = np.vstack(labels)
        return X, Y
    else:
        return np.array([]), np.array([])

In [3]:
# Load the dataset 
X_train, Y_train = load_data("spectrograms_train", "labels_train", filter_piano=False)  
X_train = np.expand_dims(X_train, axis=-1)  
X_val, Y_val = load_data("spectrograms_validation", "labels_validation", filter_piano=False)  
X_val = np.expand_dims(X_val, axis=-1)  

# Combine sets
X_train = np.concatenate([X_train, X_val], axis=0)
Y_train = np.concatenate([Y_train, Y_val], axis=0)
print("Final X_train:", X_train.shape, "Final Y_train:", Y_train.shape)

Loading Data: 100%|██████████| 270/270 [00:00<00:00, 2747.18file/s]


Final X_train: (385420, 32, 42, 1) Final Y_train: (385420, 5)


In [4]:
# Occurrences
class_counts = np.sum(Y_train, axis=0)
total_samples = len(Y_train)

for i, count in enumerate(class_counts):
    print(f"Class {i}: {count} occurrences ({count / total_samples:.2%} of the data)")

Class 0: 179228 occurrences (46.50% of the data)
Class 1: 300261 occurrences (77.90% of the data)
Class 2: 311040 occurrences (80.70% of the data)
Class 3: 188441 occurrences (48.89% of the data)
Class 4: 334015 occurrences (86.66% of the data)


In [5]:
model = Sequential([
    Input(shape=(32, 42, 1)), 
    Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),  
    Dense(5, activation='sigmoid', dtype='float32')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

I0000 00:00:1743229557.916013   14786 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 42, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 21, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 10, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 10, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        81,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,605 (412.52 KB)

 Trainable params: 105,605 (412.52 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train model
history = model.fit(X_train, Y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.1)

# Final loss & accuracy
final_loss = history.history['loss'][-1]
final_accuracy = history.history['binary_accuracy'][-1]
print(f"Final Loss: {final_loss:.4f}, Final Accuracy: {final_accuracy:.4f}")

Epoch 1/10


I0000 00:00:1743229576.313420   14944 service.cc:148] XLA service 0x7f9074005aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743229576.313520   14944 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-03-28 23:26:16.344700: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743229576.505305   14944 cuda_dnn.cc:529] Loaded cuDNN version 90300


  42/5420 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - binary_accuracy: 0.6826 - loss: 0.9551

I0000 00:00:1743229578.581034   14944 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5420/5420 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - binary_accuracy: 0.8244 - loss: 0.4618 - val_binary_accuracy: 0.8526 - val_loss: 0.3622
Epoch 2/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - binary_accuracy: 0.8678 - loss: 0.3438 - val_binary_accuracy: 0.8675 - val_loss: 0.3357
Epoch 3/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - binary_accuracy: 0.8728 - loss: 0.3317 - val_binary_accuracy: 0.8629 - val_loss: 0.3471
Epoch 4/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - binary_accuracy: 0.8749 - loss: 0.3266 - val_binary_accuracy: 0.8736 - val_loss: 0.3275
Epoch 5/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - binary_accuracy: 0.8765 - loss: 0.3235 - val_binary_accuracy: 0.8702 - val_loss: 0.3333
Epoch 6/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - binary_accuracy: 0.8771 - loss: 0.3219 - val_binary_accuracy: 0.8613 - val_loss: 0.3403
Epoch 7/10
5420/5420 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - binary_accuracy: 0.8777 - loss: 0.3199 - val_binary_accuracy: 0.8690 - val_loss: 0.3

In [7]:
# Get predictions
Y_pred = model.predict(X_train)

# Convert to binary (threshold the probabilities)
threshold = 0.5
Y_pred_binary = (Y_pred > threshold).astype(int)

# Calculate metrics
precision = precision_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
recall = recall_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
f1 = f1_score(Y_train, Y_pred_binary, average='samples', zero_division=0)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

12045/12045 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step
Precision: 0.8651
Recall: 0.8976
F1-Score: 0.8697


In [8]:
X_test, Y_test = load_data("spectrograms_test", "labels_test")  
X_test = np.expand_dims(X_test, axis=-1)  

Y_pred_test = model.predict(X_test)
Y_pred_test_binary = (Y_pred_test > 0.5).astype(int)

precision_test = precision_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
recall_test = recall_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
f1_test = f1_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)

print(f"Test Precision: {precision_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")

Loading Data: 100%|██████████| 151/151 [00:00<00:00, 1366.98file/s]


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Test Precision: 0.8733
Test Recall: 0.8881
Test F1-Score: 0.8678


In [9]:
# Compute per-class precision, recall, and F1-score
precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
    Y_test, Y_pred_test_binary, average=None, zero_division=0
)

# Occurrences
class_counts = np.sum(Y_train, axis=0)
total_samples = len(Y_train)

num_classes = Y_test.shape[1]  # Number of instrument classes
for i in range(num_classes):
    print(f"Class {i}: Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, "
          f"F1={f1_per_class[i]:.4f} | Occurrences={int(class_counts[i])} ({class_counts[i] / total_samples:.2%})")

Class 0: Precision=0.7731, Recall=0.7756, F1=0.7743 | Occurrences=179228 (46.50%)
Class 1: Precision=0.8853, Recall=0.9691, F1=0.9253 | Occurrences=300261 (77.90%)
Class 2: Precision=0.9251, Recall=0.9861, F1=0.9546 | Occurrences=311040 (80.70%)
Class 3: Precision=0.9536, Recall=0.7736, F1=0.8542 | Occurrences=188441 (48.89%)
Class 4: Precision=0.9662, Recall=0.9746, F1=0.9704 | Occurrences=334015 (86.66%)


In [19]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 32, 40, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 20, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 20, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 8, 10, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 8, 10, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 4, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 5, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 514,193 (1.96 MB)

 Trainable params: 171,397 (669.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 342,796 (1.31 MB)

In [20]:
model.save("instrument_classifier.h5")
model = tf.keras.models.load_model("instrument_classifier.h5", compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("instrument_classifier.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpuvpmf_09/assets


INFO:tensorflow:Assets written to: /tmp/tmpuvpmf_09/assets


Saved artifact at '/tmp/tmpuvpmf_09'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 40, 1), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  140103781968256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103781973360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103781960688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140104423013040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782011952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782012656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782009840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782015120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782069552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140103782067440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140104423461408

W0000 00:00:1742875013.770337     647 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742875013.770383     647 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 20:56:53.770793: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpuvpmf_09
2025-03-24 20:56:53.771591: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 20:56:53.771602: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpuvpmf_09
I0000 00:00:1742875013.777529     647 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-24 20:56:53.778843: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 20:56:53.816662: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpuvpmf_09
2025-03-24 20:56:53.828498: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 